In [1]:
# Code Cell 1
# ----Importing necessary libraries----

import os
import MDAnalysis as mda
import pandas as pd
import multiprocessing as mp
import subprocess as sp
import numpy as np
import time

from vina import Vina
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Crippen, Lipinski, rdMolDescriptors
from joblib import Parallel, delayed
from dimorphite_dl import protonate_smiles
from sascorer import calculateScore
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import CellIsRule, ColorScaleRule
from sklearn.preprocessing import MinMaxScaler
from tqdm.auto import tqdm

In [2]:
# Code Cell 2
# ----Preparing filesystem----


current_dir = os.getcwd()

data_dir = os.path.join(current_dir, "Data")
data_protein_dir = os.path.join(data_dir, "Protein")
data_ligand_dir = os.path.join(data_dir, "Ligands_Raw")
os.makedirs(data_protein_dir, exist_ok=True)
os.makedirs(data_ligand_dir, exist_ok=True)
os.makedirs(data_protein_dir, exist_ok=True)

prep_dir = os.path.join(current_dir, "Prep")
prep_protein_dir = os.path.join(prep_dir, "Proteins_prepared")
prep_ligand_dir = os.path.join(prep_dir, "Ligands_prepared")
os.makedirs(prep_dir, exist_ok=True)
os.makedirs(prep_protein_dir, exist_ok=True)
os.makedirs(prep_ligand_dir, exist_ok=True)

docking_results_dir = os.path.join(current_dir, "Docking")
docking_results_pdbqts_dir = os.path.join(docking_results_dir, "Result_PDBQTs")
docking_results_bestposes_dir = os.path.join(docking_results_dir, "BestPoses")
os.makedirs(docking_results_dir, exist_ok=True)
os.makedirs(docking_results_pdbqts_dir, exist_ok=True)
os.makedirs(docking_results_bestposes_dir, exist_ok=True)

vina_binary_path = os.path.join(current_dir,"bin", "vina")
print(f"\nFile system created. :)\
      \nPlease move your receptor's PDB file to {data_protein_dir} and your ligand SDFs to {data_ligand_dir}\
      ")

if not os.path.exists(vina_binary_path):
    raise FileNotFoundError(f"Vina binary not found in 'bin' folder :(\nPlease reinstall or check if the name of \
                            the binary has been changed or not.\nIf changed, please rename it\
                            back to 'vina'")


File system created. :)      
Please move your receptor's PDB file to /home/freax/HTVS/DDDD_Presentation_Build/Malonate_test/Data/Protein and your ligand SDFs to /home/freax/HTVS/DDDD_Presentation_Build/Malonate_test/Data/Ligands_Raw      


## Protein Preparation

In [3]:
# Code Cell 3
# ----Isolating protein structure and protonating----

pdb_id = "7jgw".upper()
pdb_file2 = os.path.join(data_protein_dir, f"{pdb_id}_2.pdb")
pdb_file1 = os.path.join(data_protein_dir, f"{pdb_id}.pdb")
pH = 7.4

u = mda.Universe(pdb_file2)
v = mda.Universe(pdb_file1)

prot_naked = u.select_atoms("protein")
# lig_only = v.select_atoms("resname V9S")
prot_naked.write(os.path.join(data_protein_dir, f"{pdb_id}_naked.pdb"))

pdb_out = os.path.join(prep_protein_dir, f"{pdb_id}_H.pdb")
pdb_in  = os.path.join(data_protein_dir, f"{pdb_id}_naked.pdb")
pqr_out = os.path.join(prep_protein_dir, f"{pdb_id}_prepared.pqr")

prepcmd = ["pdb2pqr", f"--pdb-output={pdb_out}", f"{pdb_in}", f"{pqr_out}", f"--pH={pH}", "--whitespace",
           "--ff=AMBER"]

try:
    preprun = sp.run(prepcmd, check=True, text=True)

except sp.CalledProcessError as e:
    print(f"\npdb2pqr FAILED! Return code: {e.returncode} \n")
    raise RuntimeError(f"\nProtein preparation FAILED. Terminating execution...  :( \n") from e

/home/freax/miniconda3/envs/Informatics/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: /home/freax/HTVS/DDDD_Presentation_Build/Malonate_test/Data/Protein/7JGW_naked.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up mole

In [4]:
# Code Cell 4
# ----Preparing protein PDBQT file from Meeko this time----

prot_pdbqt_out = os.path.join(prep_protein_dir, f"{pdb_id}_prepared.pdbqt")

prepprotcmd = ["mk_prepare_receptor.py",
               "-i", pdb_out,
               "--write_pdbqt", prot_pdbqt_out]

try:
    runprepprotcmd = sp.run(prepprotcmd, check=True, text=True, capture_output=True)
    print(f"\n{pdb_id} has been prepared and saved to {prot_pdbqt_out}\n")

except sp.CalledProcessError as e:
    print(f"{pdb_id} preparation failed (exit code : {e.returncode})\n")
    print(f"Error (first 200 characters): {e.stderr[:200]}\n")
    raise RuntimeError(f"\nProtein preparation failed. Terminating execution... :( \n") from e


7JGW has been prepared and saved to /home/freax/HTVS/DDDD_Presentation_Build/Malonate_test/Prep/Proteins_prepared/7JGW_prepared.pdbqt



In [5]:
# Code Cell 5
# ----Setting grid box coordinates----

gbox_centre = [-2.96, -2.55, 26.75]
gbox_size = [11.25, 20.10, 24.24]

# gbox_centre = lig_only.center_of_geometry().tolist()
# gbox_size = lig_only.positions.max(axis=0) - lig_only.positions.min(axis=0) + 10.0
# gbox_size = gbox_size.tolist()

print(f"Grid box dimensions:\nCenter: {gbox_centre}\nSize: {gbox_size}\n")

Grid box dimensions:
Center: [-2.96, -2.55, 26.75]
Size: [11.25, 20.1, 24.24]



---

## Ligand Preparation (more robust this time)

In [6]:
# Code Cell 6
# ----Listing ligand SDF files to be prepared into PDBQTs----

ligfilelist = [f for f in os.listdir(data_ligand_dir) if f.endswith('.sdf')]
print(f"{len(ligfilelist)} ligand files found in {data_ligand_dir}.\nThey are:")

for idx, lig in enumerate(ligfilelist):
    print(f"{idx + 1}. {lig}")

7 ligand files found in /home/freax/HTVS/DDDD_Presentation_Build/Malonate_test/Data/Ligands_Raw.
They are:
1. 3a.sdf
2. 4b.sdf
3. 3c.sdf
4. 4a.sdf
5. 4c.sdf
6. 5-fluorouracil.sdf
7. 3b.sdf


In [ ]:
# Code Cell 7
# ----Preparing ligand SDFs into PDBQTs----

failed_ligs = {}
for ligand in tqdm(ligfilelist, desc="🔬 Preparing ligands", dynamic_ncols=True):

    lig_path = os.path.join(data_ligand_dir, ligand)
    mol_supplier = Chem.SDMolSupplier(lig_path, removeHs=False)
    

    for mol in mol_supplier:
        if mol is None:
            print(f"Warning: Could not read molecule in file {ligand}. Skipping...")
            failed_ligs[ligand] = "Could not read molecule from file"
            continue

        if not mol.GetNumConformers():
            print(f"\n{ligand} has no 3D geometry information. 3D geometry information \
                  will be generated with ETKDG v3.")

        lig_name = os.path.splitext(ligand)[0]

        smi = Chem.MolToSmiles(mol, isomericSmiles=True)
        print(f"Converted {lig_name} to SMILES: {smi}\n\nProtonating {lig_name}...\n", flush=True)

        proton_variants = protonate_smiles(smi, ph_min=7.4, ph_max=7.4, max_variants=1, precision=1.0)

        if not proton_variants:
            print(f"Warning: No protonation variants generated for {lig_name}. Skipping...")
            failed_ligs[ligand] = "Could not generate protonation variants"
            continue

        proton_smi = proton_variants[0]
        print(f"Protonated SMILES for {lig_name}: {proton_smi}\n", flush=True)
        proton_mol = Chem.MolFromSmiles(proton_smi)
        proton_mol = Chem.AddHs(proton_mol)

        flex = AllChem.CalcNumRotatableBonds(proton_mol)
        NumberOfConformations = 10 if flex >= 6 else 3
        print(f"{lig_name} has {flex} rotatable bonds. Generating {NumberOfConformations} conformations...\n\n", flush=True)

        params = AllChem.ETKDGv3()
        ids = AllChem.EmbedMultipleConfs(proton_mol, numConfs=NumberOfConformations, params=params)
        if len(ids) == 0:
            print(f"⚠️ 3D embedding failed for {lig_name}. Skipping...", flush=True)
            failed_ligs[ligand] = "3D structure generation failed"
            continue

        props = AllChem.MMFFGetMoleculeProperties(proton_mol)
        energies = []

        for cid in ids:
            ff = AllChem.MMFFGetMoleculeForceField(proton_mol, props, confId=cid)
            ff.Minimize()
            energies.append((cid, ff.CalcEnergy()))

        best_conf = min(energies, key=lambda x: x[1])[0]
        print(f"Lowest energy conformer ID: {best_conf} with energy {min(energies, key=lambda x: x[1])[1]:.2f} kcal/mol", flush=True)
        best_mol = Chem.Mol(proton_mol)
        best_mol.RemoveAllConformers()
        best_mol.AddConformer(proton_mol.GetConformer(best_conf), assignId=True)

        Chem.MolToMolFile(best_mol, os.path.join(prep_ligand_dir, f"{lig_name}_prot.sdf"))

        lig_in  = os.path.join(prep_ligand_dir, f"{lig_name}_prot.sdf")
        lig_out = os.path.join(prep_ligand_dir, f"{lig_name}_prepared.pdbqt")

        prepligcmd = ["mk_prepare_ligand.py",
                      "-i", lig_in,
                      "-o", lig_out]

        
        try:
            runprepligcmd = sp.run(prepligcmd, check=True, text=True, capture_output=True)
            print(f"Prepared {lig_name} ligand has been saved to {prep_ligand_dir}/{lig_name}_prepared.pdbqt\n", flush=True)

        except sp.CalledProcessError as e: 
            print(f"\n{lig_name} preparation FAILED (exit code: {e.returncode}).\n \
                  This ligand will be skipped.\n")
            print(f"\nError (first 200 chars): {e.stderr[:200]}\n")
            failed_ligs[ligand] = f"PDBQT conversion failed. Failure message: {e.stderr[:200]}"
            continue
    
if failed_ligs:
    print("\n\nSome ligands failed to prepare. They are:\n")
    for flig, reason in failed_ligs.items():
        print(f"{flig}: {reason}")

🔬 Preparing ligands:   0%|          | 0/7 [00:00<?, ?it/s]

Converted 3a to SMILES: CCOC(=O)C(Cc1ccc([N+](=O)[O-])cc1)C(=O)OCC

Protonating 3a...

Protonated SMILES for 3a: CCOC(=O)C(Cc1ccc([N+](=O)[O-])cc1)C(=O)OCC

3a has 9 rotatable bonds. Generating 10 conformations...


Lowest energy conformer ID: 8 with energy 10.08 kcal/mol
Prepared 3a ligand has been saved to /home/freax/HTVS/DDDD_Presentation_Build/Malonate_test/Prep/Ligands_prepared/3a_prepared.pdbqt

Converted 4b to SMILES: COC(=O)C(Cc1ccc(N)cc1)C(=O)OC

Protonating 4b...

Protonated SMILES for 4b: COC(=O)C(Cc1ccc(N)cc1)C(=O)OC

4b has 7 rotatable bonds. Generating 10 conformations...


Lowest energy conformer ID: 9 with energy -28.74 kcal/mol
Prepared 4b ligand has been saved to /home/freax/HTVS/DDDD_Presentation_Build/Malonate_test/Prep/Ligands_prepared/4b_prepared.pdbqt

Converted 3c to SMILES: [H]OC(=O)C(Cc1ccc([N+](=O)[O-])cc1)C(=O)O[H]

Protonating 3c...

Protonated SMILES for 3c: O=C([O-])C(Cc1ccc([N+](=O)[O-])cc1)C(=O)[O-]

3c has 5 rotatable bonds. Generating 3 conformations

---

## Screening the ligands with `vina` binary and parallelization

In [10]:
# Code Cell 8
# ----Listing ligand PDBQT files to be screened----


lig_pdbqt_list = [f for f in os.listdir(prep_ligand_dir) if f.endswith('.pdbqt')]
print(f"{len(lig_pdbqt_list)} prepared ligands found in {prep_ligand_dir}.\nThey are:")
for idx, lig in enumerate(lig_pdbqt_list):
    print(f"{idx + 1}. {lig}")

7 prepared ligands found in /home/freax/HTVS/DDDD_Presentation_Build/Malonate_test/Prep/Ligands_prepared.
They are:
1. 4c_prepared.pdbqt
2. 3b_prepared.pdbqt
3. 4b_prepared.pdbqt
4. 5-fluorouracil_prepared.pdbqt
5. 3a_prepared.pdbqt
6. 4a_prepared.pdbqt
7. 3c_prepared.pdbqt


In [ ]:
# Code Cell 9
# ----Screening----


def screen_binary(inp_lig_filename: str, exhaustiveness: int, num_poses: int, protPDBID: str = pdb_id,
           gridbox_center: list = gbox_centre, gridbox_size: list = gbox_size) -> dict[str, str|float]:
    """
    Perform molecular docking using the AutoDock Vina command-line binary and extract the best docking score.

    This function executes AutoDock Vina as an external subprocess to dock a prepared ligand (PDBQT format)
    against a prepared receptor structure. It streams live docking output to both the terminal and a log file,
    parses the top docking score from the resulting output PDBQT, and returns key screening results in a 
    dictionary. In case of failure, detailed error information is captured and returned instead.

    Args:
        inp_lig_filename (str): Filename of the prepared ligand (e.g., `"ligand1_prepared.pdbqt"`), 
            expected to be located in `prep_ligand_dir`.
        exhaustiveness (int): Docking exhaustiveness parameter that controls the thoroughness 
            of the search; higher values increase accuracy at the cost of runtime.
        num_poses (int): Number of docking poses (`--num_modes`) to generate and save.
        protPDBID (str, optional): PDB ID of the prepared target protein. 
            Defaults to the global variable `pdb_id`.
        gridbox_center (list, optional): Center of the docking grid box as `[x, y, z]` coordinates.
            Defaults to the global variable `gbox_centre`.
        gridbox_size (list, optional): Size of the docking grid box as `[x, y, z]` dimensions.
            Defaults to the global variable `gbox_size`.

    Returns:
        dict[str, str | float]: A dictionary containing one of the following outcomes:

            **Successful Docking**
            - `"Ligand Screened"` (str): The ligand name (without the `_prepared.pdbqt` suffix).
            - `"Best Docking Score"` (float): The lowest binding energy (in kcal/mol) 
              found among generated poses.
            - `"Docking time (s)"` (float): Total docking runtime in seconds.

            **Failed Docking**
            - `"Ligand not Screened"` (str): The ligand name.
            - `"Error"` (str): Error message extracted from Vina’s output or exception trace.

    Raises:
        subprocess.CalledProcessError: Raised internally when AutoDock Vina exits with a non-zero return code.
            This exception is caught and converted into a user-friendly error dictionary.

    Side Effects:
        - Writes a ligand-specific `.log` file containing Vina’s full console output to `docking_results_pdbqts_dir`.
        - Generates a `.pdbqt` file containing the docked poses in the same directory.
        - Prints live progress lines from Vina to standard output (helpful for Jupyter or terminal tracking).

    Example:
        >>> result = screen_binary("lig1_prepared.pdbqt", exhaustiveness=8, num_poses=10, protPDBID="4EY7")
        >>> print(result)
        {'Ligand Screened': 'lig1', 'Best Docking Score': -9.1, 'Docking time (s)': 52.47}

        >>> result = screen_binary("badlig_prepared.pdbqt", 8, 10, "4EY7")
        >>> print(result)
        {'Ligand not Screened': 'badlig', 'Error': 'No PDBQT or empty PDBQT output generated. Please check ...'}
    """
    ligname = inp_lig_filename.strip().split("_prepared.pdbqt")[0]
    inp_ligpath = os.path.join(prep_ligand_dir, inp_lig_filename)
    outpath = os.path.join(docking_results_pdbqts_dir, f"{ligname}-{protPDBID}_docked.pdbqt")
    logpath = os.path.join(docking_results_pdbqts_dir, f"{ligname}-{protPDBID}_docked.log")
    

    cmd = [
        vina_binary_path,
        "--receptor", prot_pdbqt_out,
        "--ligand", inp_ligpath,
        "--center_x", str(gridbox_center[0]),
        "--center_y", str(gridbox_center[1]),
        "--center_z", str(gridbox_center[2]),
        "--size_x", str(gridbox_size[0]),
        "--size_y", str(gridbox_size[1]),
        "--size_z", str(gridbox_size[2]),
        "--exhaustiveness", str(exhaustiveness),
        "--num_modes", str(num_poses),
        "--out", outpath,
        "--cpu", "2",
        "--verbosity", "1"
    ]

    start = time.time()

    try:
        captured_lines = []

        # ---- Run Vina and stream output ----
        with open(logpath, "w+", encoding="utf-8") as logfile:
            process = sp.Popen(cmd, text=True, stdout=sp.PIPE, stderr=sp.STDOUT)

            for line in process.stdout:
                # print(f"[{ligname}] {line}", end="")    # Live output into Jupyter Notebook's cell output
                print(line, file=logfile, end="")         # Writing line into ligand's log
                captured_lines.append(line)
            
            process.wait()
        
        # ---- Handle failed docks ----
        if process.returncode != 0: 
            error_text = "".join(captured_lines[-30:])  # Usually the last few lines contain the error
            raise sp.CalledProcessError(process.returncode, cmd, output=error_text)
        
        # ---- Parse best docking score of the ligand from its output PDBQT file ----
        best_score = None
        Bmodel1found = False

        if not os.path.exists(outpath) or os.path.getsize(outpath) == 0:
            return {"Ligand not Screened": ligname,
                    "Error": f"No PDBQT or empty PDBQT output generated. Please check {docking_results_pdbqts_dir}"}
        
        with open(outpath, "r", encoding="utf-8") as f:
            for line in f:
                line  = line.strip().upper()

                if line.startswith("MODEL"):
                    parts = line.split()
                    if len(parts) >= 2 and parts[1] == "1":
                        Bmodel1found = True
                        continue


                if Bmodel1found and line.startswith("ENDMDL"):
                    break


                if Bmodel1found and line.startswith("REMARK VINA RESULT:"):
                    tokens = line.split()
                    try:
                        best_score = float(tokens[3]) if tokens[2] == "RESULT:" else float(tokens[2])
                        break
                    except (ValueError, IndexError):
                        continue
                    
        
        if best_score is None:
            return {"Ligand not Screened": ligname,
                    "Error": f"No valid score found in {ligname}-{protPDBID}_docked.pdbqt. Please check \
                        the PDBQT file for any issues."}
        
        elapsed = round(time.time() - start, 2)

        return {"Ligand Screened": ligname,
                "Best Docking Score (kcal/mol)": best_score,
                "Docking time (s)": elapsed}
    
    except sp.CalledProcessError as e:
        err_msg = e.output.strip() if e.output else f"Exit code {e.returncode}"
        return {"Ligand not Screened": ligname,
                "Error": err_msg}

total_cpu = (mp.cpu_count()) // 2 
safe_jobs = ((total_cpu - 2) // 2)

all_results = Parallel(n_jobs=safe_jobs, verbose=15)(delayed(screen_binary)(lig, 16, 10)
                                                     for lig in lig_pdbqt_list)

results_list = [d for d in all_results if "Best Docking Score (kcal/mol)" in d]
errs_list = [d for d in all_results if "Error" in d]



In [ ]:
# Code Cell 10
# ---- Descriptor Calculation & Property Profiling----

summary_output = os.path.join(docking_results_dir, f"{pdb_id}_screening_results_summary.xlsx")
if results_list:
        results_df = pd.DataFrame(results_list)
        results_df = results_df.sort_values(by="Best Docking Score (kcal/mol)").reset_index(drop=True)
        results_df["Rank"] = range(1, len(results_df) + 1)
        results_df = results_df[["Rank", "Ligand Screened", "Best Docking Score (kcal/mol)", "Docking time (s)"]]


desc_records = []

for idx, rows in results_df.iterrows():
    ligand_name = rows["Ligand Screened"]
    ligand_sdf = os.path.join(prep_ligand_dir, f"{ligand_name}_prot.sdf")

    if not os.path.exists(ligand_sdf):
        print(f"⚠️ Skipping {ligand_name}: no prepared SDF found.")
        continue

    mol = Chem.MolFromMolFile(ligand_sdf)
    if mol is None:
        print(f"⚠️ Could not read {lig_name} into RDKit. Skipping.")
        continue

    BDS = results_df.loc[idx, "Best Docking Score (kcal/mol)"]
    MW = Descriptors.MolWt(mol)
    logP = Crippen.MolLogP(mol)
    HBD = Lipinski.NumHDonors(mol)
    HBA = Lipinski.NumHAcceptors(mol)
    TPSA = rdMolDescriptors.CalcTPSA(mol)
    SAScore = calculateScore(mol)
    HeavyAtoms = mol.GetNumHeavyAtoms()
    nRot = AllChem.CalcNumRotatableBonds(mol)
    LE = abs(BDS) / HeavyAtoms

    Lipinski_Pass = "✅" if (MW < 500) and (logP < 5) and (HBD <= 5) and (HBA <= 10) else "❌"
    Veber_Pass = "✅" if (nRot <= 10) and (TPSA <= 140) else "❌"

    desc_records.append({"Ligand Screened": ligand_name,
                         "Best Docking Score (kcal/mol)": BDS,
                         "Mol Wt.": MW,
                         "logP": logP,
                         "HBD": HBD,
                         "HBA": HBA,
                         "TPSA": TPSA,
                         "Num. Rotatable Bonds":nRot,
                         "SA_Score": SAScore,
                         "Ligand Efficiency": LE,
                         "Lipinski's Rule of 5": Lipinski_Pass,
                         "Veber's Rule": Veber_Pass})

desc_df = pd.DataFrame(desc_records)
desc_df = desc_df.sort_values(by="Best Docking Score (kcal/mol)")
desc_df["Rank"] =  range(1, len(desc_df) + 1)
desc_df = desc_df[
    [
        "Rank",
        "Ligand Screened",
        "Best Docking Score (kcal/mol)",
        "Ligand Efficiency",
        "SA_Score",
        "Mol Wt.",
        "logP",
        "HBD",
        "HBA",
        "Lipinski's Rule of 5",
        "TPSA",
        "Num. Rotatable Bonds",
        "Veber's Rule",
    ]
]

desc_df


In [ ]:
# Code Cell 11
# ----Composite Scoring and Ranking----


scaler = MinMaxScaler()


# Normalize docking score (inverted because lower is better)
desc_df["Docking_Norm"] = scaler.fit_transform(-desc_df[["Best Docking Score (kcal/mol)"]])

# Normalize ligand efficiency (higher is better)
desc_df["LE_Norm"] = scaler.fit_transform(desc_df[["Ligand Efficiency"]])

# Normalize SA Score (lower is better therefore inverse)
desc_df["SA_Norm"] = scaler.fit_transform(desc_df[["SA_Score"]])

# Penalize extreme SA molecules (the difficult to synthesise ones) exponentially
desc_df["SA_Penalty"] = np.exp(0.15 * (desc_df["SA_Score"] - 5))

# Composite Score Calculation
DockWt, LEWt, SAWt, SAPen = 0.45, 0.30, 0.15, 0.10

def compute_bonus(row: str) -> float:
    bonus = 0

    if row["Lipinski's Rule of 5"] == "✅":
        bonus += 0.05
    
    if row["Veber's Rule"] == "✅":
        bonus += 0.05
    
    return bonus

desc_df["Composite Score"] = (
    (DockWt * desc_df["Docking_Norm"]) +
    (LEWt * desc_df["LE_Norm"]) -
    (SAWt * desc_df["SA_Norm"]) -
    (SAPen * desc_df["SA_Penalty"]) +
    desc_df.apply(compute_bonus, axis=1)
)

desc_df = desc_df.sort_values(by="Composite Score", ascending=False).reset_index(drop=True)
desc_df["Rank"] = range(1, len(desc_df) + 1)

desc_df = desc_df[
    [
        "Rank",
        "Ligand Screened",
        "Best Docking Score (kcal/mol)",
        "Ligand Efficiency",
        "SA_Score",
        "Mol Wt.",
        "logP",
        "HBD",
        "HBA",
        "Lipinski's Rule of 5",
        "TPSA",
        "Num. Rotatable Bonds",
        "Veber's Rule",
        "Composite Score"
    ]
]

desc_df

In [ ]:
# Code Cell 12
# ----Generation of summary Excel spreadsheet with additional formatting----


with pd.ExcelWriter(summary_output, engine="openpyxl") as writer:
    desc_df.to_excel(writer, index=False, sheet_name="Docking + Descriptors")
    if errs_list:
        errs_df = pd.DataFrame(errs_list)
        errs_df.to_excel(writer, sheet_name="Errors")

print(f"🧪 Descriptor sheet added to {summary_output}")
print(f"Columns: {list(desc_df.columns)}")

# Load workbook and target sheet
wb = load_workbook(summary_output)
ws = wb["Docking + Descriptors"]

# Docking Score gradient (C column)
gradient = ColorScaleRule(
    start_type="min", start_color="63BE7B",    # green
    mid_type="percentile", mid_value=50, mid_color="D7F930",  # yellow
    end_type="max", end_color="F8696B"         # red
)
ws.conditional_formatting.add(f"C2:C{ws.max_row}", gradient)

# Rule-of-Five Violations Highlight
fail_fill = PatternFill(start_color="FC4F4F", end_color="FC4F4F", fill_type="solid")  # light red

# Column mappings
col_map = {
    "Mol Wt.": "F",
    "logP": "G",
    "HBD": "H",
    "HBA": "I"
}

# MW ≥ 500 → highlight
ws.conditional_formatting.add(
    f"{col_map['Mol Wt.']}2:{col_map['Mol Wt.']}{ws.max_row}",
    CellIsRule(operator="greaterThanOrEqual", formula=["500"], fill=fail_fill)
)

# logP ≥ 5 → highlight
ws.conditional_formatting.add(
    f"{col_map['logP']}2:{col_map['logP']}{ws.max_row}",
    CellIsRule(operator="greaterThanOrEqual", formula=["5"], fill=fail_fill)
)

# HBD > 5 → highlight
ws.conditional_formatting.add(
    f"{col_map['HBD']}2:{col_map['HBD']}{ws.max_row}",
    CellIsRule(operator="greaterThan", formula=["5"], fill=fail_fill)
)

# HBA > 10 → highlight
ws.conditional_formatting.add(
    f"{col_map['HBA']}2:{col_map['HBA']}{ws.max_row}",
    CellIsRule(operator="greaterThan", formula=["10"], fill=fail_fill)
)


# Save workbook
wb.save(summary_output)
print("🎯 RO5 descriptor highlighting added successfully!")


---

In [ ]:
# Code Cell 13
# ----Listing PDBQTs for best pose generation----

ligresult_list = [f for f in os.listdir(docking_results_pdbqts_dir) if f.endswith('.pdbqt')]
print(f"{len(ligresult_list)} docking result files found in {docking_results_pdbqts_dir}.They are:\n")
for ligr in ligresult_list:
    print(f"{ligresult_list.index(ligr)+1}. {ligr}")

In [ ]:
# Code Cell 14
# ----Generating PDBs of the best poses of each ligand complexed with the protein----

failed_extractions = {}

for docked_lig in ligresult_list:
    docked_lig_name = docked_lig.split(f"-{pdb_id}_docked.pdbqt")[0]
    
    docked_pdbqt_path = os.path.join(docking_results_pdbqts_dir, docked_lig)
    docked_tmp_path = os.path.join(docking_results_dir, f"{docked_lig_name}_MODEL1_tmp.pdbqt")
    docked_pdb_ligpath = os.path.join(docking_results_dir, f"{docked_lig_name}_bestpose.pdb")
    docked_pdb_complexpath = os.path.join(docking_results_bestposes_dir, f"{docked_lig_name}_{pdb_id}_bestpose.pdb")
    found_model = False

    try:
        # ---- Extract MODEL 1 ----
        with open(docked_pdbqt_path, "r", encoding="utf-8") as infile, open(docked_tmp_path, "w", encoding="utf-8") as outfile:
            keep = False
            for line in infile:
                token = line.strip().upper()

                if token.startswith("MODEL"):
                    parts = token.split()
                    if len(parts) >= 2 and parts[1] == "1":
                        keep = True
                        found_model = True
                        continue

                if keep and token.startswith("ENDMDL"):
                    break

                if keep:
                    outfile.write(line)

        if not found_model:
            failed_extractions[docked_lig] = "MODEL 1 not found in docking output"
            if os.path.exists(docked_tmp_path):
                os.remove(docked_tmp_path)
            continue

        if not os.path.exists(docked_tmp_path) or os.path.getsize(docked_tmp_path) == 0:
            failed_extractions[docked_lig] = "Temporary MODEL 1 file missing or empty"
            if os.path.exists(docked_tmp_path):
                os.remove(docked_tmp_path)
            continue

        # ---- Convert MODEL 1 PDBQT → PDB using Open Babel ----
        obabel_cmd = ["obabel", "-ipdbqt", docked_tmp_path, "-opdb", "-O", docked_pdb_ligpath]
        try:
            result = sp.run(obabel_cmd, text=True, check=True, capture_output=True)
        except sp.CalledProcessError as e:
            err_output = e.stderr.strip() if e.stderr else "No stderr captured"
            failed_extractions[docked_lig] = f"Open Babel conversion failed: {err_output[:200]}"
            os.remove(docked_tmp_path)
            continue

        # ---- Validate PDB output ----
        if not os.path.exists(docked_pdb_ligpath) or os.path.getsize(docked_pdb_ligpath) == 0:
            failed_extractions[docked_lig] = "Open Babel created empty or invalid PDB file"
            os.remove(docked_tmp_path)
            if os.path.exists(docked_pdb_ligpath):
                os.remove(docked_pdb_ligpath)
            continue

        # ---- Merge best pose with receptor ----
        try:
            prot_naked = mda.Universe(os.path.join(data_protein_dir, f"{pdb_id}_naked.pdb"))
            lig_best = mda.Universe(docked_pdb_ligpath)
            complex_bestpose = mda.Merge(prot_naked.atoms, lig_best.atoms)
            complex_bestpose.atoms.write(docked_pdb_complexpath)
            os.remove(docked_pdb_ligpath)

            print(f"✅ Best pose for {docked_lig_name} saved to {docked_pdb_complexpath}\n", flush=True)

        except Exception as e:
            failed_extractions[docked_lig] = f"Failed to merge ligand and receptor: {str(e)[:200]}"
            problem_dir = os.path.join(docking_results_dir, "Problem_Files")
            os.makedirs(problem_dir, exist_ok=True)

            try:
                if os.path.exists(docked_tmp_path):
                    os.replace(docked_tmp_path, os.path.join(problem_dir, os.path.basename(docked_tmp_path)))
                if os.path.exists(docked_pdb_ligpath):
                    os.replace(docked_pdb_ligpath, os.path.join(problem_dir, os.path.basename(docked_pdb_ligpath)))
            except Exception:
                pass
            continue

    finally:
        if os.path.exists(docked_tmp_path):
            os.remove(docked_tmp_path)
        if os.path.exists(docked_pdb_ligpath):
            os.remove(docked_pdb_ligpath)


# ---- Summary of failed extractions ----
summary_output = os.path.join(docking_results_dir, f"{pdb_id}_screening_results_summary.xlsx")

if failed_extractions:
    print("\n\n⚠️ Some best-pose extractions failed:\n")
    for lig, reason in failed_extractions.items():
        print(f"{lig}: {reason}")

    failed_pose_df = pd.DataFrame(
        list(failed_extractions.items()), columns=["Ligand", "Reason"]
    )

    # Append as a new sheet in the same Excel file
    with pd.ExcelWriter(summary_output, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        failed_pose_df.to_excel(writer, index=False, sheet_name="Pose Extraction Errors")

    print(f"\n🧾 Summary appended to: {summary_output} → Sheet: 'Pose Extraction Errors'\n")

else:
    print("\n🎉 All best poses extracted and merged successfully!\n")

